<a href="https://colab.research.google.com/github/ERPGulf/changai/blob/hyrin/S1_ROBERTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!git clone --branch hyrin https://github.com/ERPGulf/changai.git

Cloning into 'changai'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 73 (delta 14), reused 73 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (73/73), 11.76 MiB | 15.07 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
import json
with open('/content/drive/MyDrive/Changai/S1/Datasets/roberto_s1.json') as f:
    raw_data = json.load(f)

In [ ]:
from datasets import Dataset
raw_data = Dataset.from_list(raw_data)
split_dataset = raw_data.train_test_split(test_size=0.2, seed=42)

In [ ]:
train_dataset = split_dataset['train']
val_dataset = split_dataset['test']

print("Train size:", len(train_dataset))
print("Validation size:", len(val_dataset))

Train size: 15371
Validation size: 3843


In [ ]:
# Save train
train_dataset.to_json('/content/sample_data/train_data.json')

# Save validation
val_dataset.to_json('/content/sample_data/val_data.json')


Creating json from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

683985

In [ ]:
# Remove 'instruction' from both datasets
train_dataset = train_dataset.remove_columns(['instruction'])
val_dataset = val_dataset.remove_columns(['instruction'])

In [ ]:
print(train_dataset.column_names)

['input', 'output']


In [ ]:
doctypes = sorted(list(set([record['output'] for record in raw_data])))

label2id = {label: idx for idx, label in enumerate(doctypes)}
id2label = {idx: label for label, idx in label2id.items()}
def encode_labels(example):
    example['label'] = label2id[example['output']]
    return example

train_dataset = train_dataset.map(encode_labels)
val_dataset = val_dataset.map(encode_labels)
print(doctypes)

Map:   0%|          | 0/15371 [00:00<?, ? examples/s]

Map:   0%|          | 0/3843 [00:00<?, ? examples/s]

['About Us Settings', 'About Us Team Member', 'Access Log', 'Account', 'Account Closing Balance', 'Accounting Dimension', 'Accounting Dimension Detail', 'Accounting Dimension Filter', 'Accounting Period', 'Accounts Receivable', 'Accounts Settings', 'Activity Cost', 'Activity Log', 'Activity Type', 'Additional Salary', 'Address', 'Address Template', 'Advance Payment Ledger Entry', 'Advance Tax', 'Advance Taxes and Charges', 'Allowed Dimension', 'Allowed To Transact With', 'Amended Document Naming Settings', 'Applicable On Account', 'Appointment', 'Appointment Booking Settings', 'Appointment Booking Slots', 'Approval', 'Asset', 'Asset Activity', 'Asset Capitalization', 'Asset Capitalization Asset Item', 'Asset Capitalization Service Item', 'Asset Capitalization Stock Item', 'Asset Capitalization, Company', 'Asset Capitalization, Payment Entry Reference', 'Asset Capitalization, Process Payment Reconciliation Log', 'Asset Capitalization, Process Payment Reconciliation Log, Company', 'Asset

In [ ]:
import csv

# Save to CSV format
with open("doctype_mapping.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Doctype", "ID"])  # Header
    for label, idx in label2id.items():
        writer.writerow([label, idx])

print("Saved as doctype_mapping.csv")

Saved as doctype_mapping.csv


In [ ]:
# As for now we are not working on the multi doctypes questions, roberto can be trained on that case later to predict multi labels.
def is_single_doctype(example):
    output = example['output']
    # If output is a list (bad), or output contains ',' or ' and '
    if isinstance(output, list):
        return False
    if ',' in output or ' and ' in output.lower():
        return False
    return True

# Filter train and val datasets
train_dataset = train_dataset.filter(is_single_doctype)
val_dataset = val_dataset.filter(is_single_doctype)

# Check sizes after cleaning
print("Train size after cleaning:", len(train_dataset))
print("Validation size after cleaning:", len(val_dataset))

Filter:   0%|          | 0/15209 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3803 [00:00<?, ? examples/s]

Train size after cleaning: 14834
Validation size after cleaning: 3690


In [ ]:
train_dataset[4:9]

{'input': ['Are there customers without a customer name field filled in?',
  'Where do I configure the SMS gateway parameters in ERPNext?',
  'What user group has access to this stage of the workflow?',
  'How do I make sure that resolution time is optional in service level settings in ERPNext?',
  'Is there a way to add a monthly recurring package for products?'],
 'output': ['Process Statement Of Accounts Customer',
  'SMS Settings',
  'Workflow Action Permitted Role',
  'Service Level Priority',
  'Subscription Plan'],
 'label': [506, 601, 794, 634, 674]}

In [ ]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def preprocess_function(examples):
    return tokenizer(examples['input'], truncation=True, padding="max_length")

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/15371 [00:00<?, ? examples/s]

Map:   0%|          | 0/3843 [00:00<?, ? examples/s]

In [ ]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install -U transformers

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Changai/S1/Model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    save_strategy="epoch",
    report_to="none"
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

Epoch,Training Loss,Validation Loss
1,6.436900,5.205587
2,4.942500,4.100965
3,3.930000,3.425646
4,3.317900,3.060697
5,3.019200,2.937833


{'eval_loss': 2.937833070755005,
 'eval_runtime': 25.4507,
 'eval_samples_per_second': 150.998,
 'eval_steps_per_second': 9.469,
 'epoch': 5.0}

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Changai/S1/Model")
tokenizer.save_pretrained("/content/drive/MyDrive/Changai/S1/Model")

('/content/drive/MyDrive/Changai/S1/Model/tokenizer_config.json',
 '/content/drive/MyDrive/Changai/S1/Model/special_tokens_map.json',
 '/content/drive/MyDrive/Changai/S1/Model/vocab.json',
 '/content/drive/MyDrive/Changai/S1/Model/merges.txt',
 '/content/drive/MyDrive/Changai/S1/Model/added_tokens.json')

In [1]:
from huggingface_hub import login

login()

In [2]:
from huggingface_hub import create_repo

create_repo("text2frappe-s1", private=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RepoUrl('https://huggingface.co/hyrinmansoor/text2frappe-s1', endpoint='https://huggingface.co', repo_type='model', repo_id='hyrinmansoor/text2frappe-s1')

In [4]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id="hyrinmansoor/text2frappe-s1",
    folder_path="/content/drive/MyDrive/Changai/S1/Model",
    path_in_repo=".",  # root of the model repo
    repo_type="model"
)


Upload 11 LFS files:   0%|          | 0/11 [00:00<?, ?it/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

...

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hyrinmansoor/text2frappe-s1/commit/17115532a2c477f314b74239f260d541f506e2c4', commit_message='Upload folder using huggingface_hub', commit_description='', oid='17115532a2c477f314b74239f260d541f506e2c4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hyrinmansoor/text2frappe-s1', endpoint='https://huggingface.co', repo_type='model', repo_id='hyrinmansoor/text2frappe-s1'), pr_revision=None, pr_num=None)

In [9]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

model_name = "hyrinmansoor/text2frappe-s1"  # can be swapped anytime
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Now you can call through API Inference also.
#API_URL = "https://api-inference.huggingface.co/models/your-username/text2frappe-s1"

In [ ]:
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification
# Change the model_path to the directory where the model was actually saved
model_path = "/content/drive/MyDrive/Changai/S1/Model"

# Add local_files_only=True to explicitly load from the local path
model = RobertaForSequenceClassification.from_pretrained(model_path, local_files_only=True)
tokenizer = RobertaTokenizerFast.from_pretrained(model_path, local_files_only=True)

In [ ]:
import torch
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test_data = [
    {"question": "Where can I view all active sales invoices?", "real_answer": "Sales Invoice"},
    {"question": "How do I create a new quotation for a customer?", "real_answer": "Quotation"},
    {"question": "List all delivery notes that are pending submission.", "real_answer": "Delivery Note"},
    {"question": "Fetch all journal entries created last month.", "real_answer": "Journal Entry"},
    {"question": "Show me approved purchase receipts from supplier ABC.", "real_answer": "Purchase Receipt"},
    {"question": "Where can I check employee attendance records?", "real_answer": "Attendance"},
    {"question": "Retrieve all leave applications pending approval.", "real_answer": "Leave Application"},
    {"question": "Find stock entries made for production today.", "real_answer": "Stock Entry"},
    {"question": "Display open support tickets for customers.", "real_answer": "Issue"},
    {"question": "Where can I configure default buying policies?", "real_answer": "Buying Settings"},
    {"question": "List all active customer subscriptions.", "real_answer": "Subscription"},
    {"question": "Show all approved supplier quotations.", "real_answer": "Supplier Quotation"},
    {"question": "Fetch submitted payment entries from this week.", "real_answer": "Payment Entry"},
    {"question": "How to track production planning orders?", "real_answer": "Production Plan"},
    {"question": "View all assets that are under maintenance.", "real_answer": "Asset Maintenance"},
]

id2label = {str(k): v for k, v in id2label.items()}

results = []

for record in test_data:
    test_question = record["question"]
    real_answer = record["real_answer"]

    inputs = tokenizer(test_question, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = logits.argmax(dim=-1).item()

    predicted_doctype = id2label[str(predicted_class_id)]

    # Store
    results.append({
        "Question": test_question,
        "Real Answer": real_answer,
        "Model Prediction": predicted_doctype,
        "Correct?": "✅" if predicted_doctype == real_answer else "❌"
    })
df_results = pd.DataFrame(results)
print(df_results)

                                             Question         Real Answer  \
0         Where can I view all active sales invoices?       Sales Invoice   
1     How do I create a new quotation for a customer?           Quotation   
2   List all delivery notes that are pending submi...       Delivery Note   
3       Fetch all journal entries created last month.       Journal Entry   
4   Show me approved purchase receipts from suppli...    Purchase Receipt   
5      Where can I check employee attendance records?          Attendance   
6   Retrieve all leave applications pending approval.   Leave Application   
7       Find stock entries made for production today.         Stock Entry   
8         Display open support tickets for customers.               Issue   
9      Where can I configure default buying policies?     Buying Settings   
10            List all active customer subscriptions.        Subscription   
11             Show all approved supplier quotations.  Supplier Quotation   